In [2]:
import os
from datetime import datetime
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import random_split, Dataset, DataLoader
from PIL import Image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


### TODO
- Fine tuning dei parametri ---> optuna
- Capire come salvare i dati [(documentazione)](https://docs.pytorch.org/tutorials/beginner/saving_loading_models.html) FATTO credo
- mapping delle probabilità da aggiungere alla rete. 

In [3]:
#Change here if you want to use RGB images instead of grayscale
is_rgb=False
#Change here the size of the crop (original image is 424)
csize=324 

In [4]:
class GalaxyJungle(Dataset): # sarebbe interessante implementare un rescale/crop
    
    #the init function initializes the directory containing the image,
    #the annotations file,
    #and both transforms
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform


    #returns number of samples in the dataset
    def __len__(self):
        return (self.img_labels).shape[0]

    #loads a sample from the dataset
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, str(self.img_labels.iloc[idx, 0])) + '.jpg'
        #retrieves the image
        image = Image.open(img_path)
        if not is_rgb: image = image.convert('L')
        #retrieves corresponding label
        label = self.img_labels.iloc[idx, 1:]
        #if possible, transform the image and the label into a tensor.
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(label.values, dtype=torch.float32)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label, self.img_labels.iloc[idx, 0]
    

transfs = transforms.Compose([
    transforms.ToTensor(), #fa già la normalizzazione se l'immagine non è un tensore
    # sarebbe interessante implementare un random crop prima del center crop per decentrare un poco le immagini????
    transforms.RandomHorizontalFlip(), # horizontal flip
    transforms.RandomVerticalFlip(), # vertical flip
    transforms.CenterCrop(csize)          #CROP
    ]) #transforms.compose per fare una pipe di transformazioni



DS = GalaxyJungle('../data/training/training_solutions_rev1.csv', '../data/training/', transfs)
training, test = random_split(DS, [.8, .2])
train_loader = DataLoader(training, batch_size=(batch_size_train := 128), shuffle=True, num_workers=8)
test_loader = DataLoader(test, batch_size=(batch_size_test := 128), shuffle=False, num_workers=8)

img, lab, indx = DS.__getitem__(0)
#print(lab)
#print(img)         #3D TENSOR    
if is_rgb:
    fig, ax = plt.subplots(1,3, figsize=(24,7))
    color = ['Reds', 'Greens', 'Blues']
    for i,j in enumerate(img):
        ax[i].imshow(j, cmap=color[i])
else:
    fig, ax = plt.subplots(1,1, figsize=(24,7))
    ax.imshow(img[0], cmap='magma')
#print(img.shape)


## Neural Net

In [5]:
#ENVIRONMENTAL VARIABLES
if is_rgb: in_channels=3
else: in_channels=1

#change here for fine tuning
kernel_size=5
out_channels=6
feature_map_2=16
max_pool_kernel=2

#for a 2 layer CNN:
size1=((csize-kernel_size)/1) + 1 #first convolution
size2=size1/max_pool_kernel       #first pooling
size3=((size2-kernel_size)/1)+1   #second convolution
size4=int(size3/max_pool_kernel)  #second pooling


In [ ]:
class GalaxyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels,out_channels,kernel_size) # convolutional layer # input 3 input channel, some output chans(feature maps) that are result from applying the kernel, also the kernel is trained during the process.
        ## Operation:  ((424-5)/1) +1 (6,420,420)
        self.pool = nn.MaxPool2d(max_pool_kernel,max_pool_kernel) #maxpool layer. divido per 2, -> (6,210,210)
        self.conv2 = nn.Conv2d(out_channels,feature_map_2,kernel_size) # 210-5 +1 = 206 -> (16,206,206) # dopo qui si fa di nuovo il pooling per cui si arriva a (16,103,103)
        self.fc1 = nn.Linear(feature_map_2*size4*size4,120) # fc è fully connected, #120 neuroni che prendono l'output
        self.fc2 = nn.Linear(120,84)# un altro fc layer che prende dai 120 neuroni e connette a 84 neuroni
        self.fc3 = nn.Linear(84,37)# idem sopra ma con 84 e 37 che è il numero di classi
        #i numeri non vincolati sono il primo 6 e il primo 16 e poi i numeri di neuroni
        ## Instance variables:
        self.loss_dict = { 'batch' : [], 'epoch' : [], 'vbatch' : [], 'vepoch' : []}
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # step 1 in cui c'è prima convoluzione e primo poool
        x = self.pool(F.relu(self.conv2(x))) # secondo step
        x = torch.flatten(x,1) # flatten operation -> 1 dimensional
        x = F.relu(self.fc1(x)) # apply relu al'output dei fully connected
        x = F.relu(self.fc2(x)) # idem sopra
        x = self.fc3(x) # output di fc3, 37 neuroni -> 37 classi ideally
        return x
    
    
    def log_the_loss(self, item,epoch=False): # per avere una history della loss???
        train = gnet.__getstate__()['training']
        if epoch and train:
            self.loss_dict['epoch'].append(item) ### get state of the model so you can ditch the validation parameter
        elif not epoch and train:
            self.loss_dict['batch'].append(item)
        elif not train and epoch:
            self.loss_dict['vepoch'].append(item)
        elif not train and not epoch:
            self.loss_dict['vbatch'].append(item)
        return item
    

gnet = GalaxyNet().to(device)


In [7]:
loss_function = nn.MSELoss()
optimizer = optim.SGD(gnet.parameters(), lr =0.001, momentum = 0.9)


In bianco e nero: 1 epoch 1m3s, loss 0.028

Colori: 1 epoch 1m37s, loss 0.029

## Traininig + validation

In [8]:
def one_epoch_train(verbose=False):
    running_loss = 0.
    last_loss = 0.
    gnet.train(True)
    for i, data in enumerate(train_loader):
        inputs, labels, idx = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = gnet(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        RMSEloss =  np.sqrt(loss.item())
        if verbose and i%10==0: print(f'Batch {i+1}/{len(train_loader)} - Loss: {RMSEloss:.3f}')
        running_loss += RMSEloss
        gnet.log_the_loss(RMSEloss, epoch=False, validation=False)
        if i == len(train_loader)-1:
            epochmean_loss = running_loss / len(train_loader)
            if verbose: print(f"---\nEpoch {epoch} - Loss: {epochmean_loss:.3f}\n---")
            gnet.log_the_loss(epochmean_loss, epoch=True, validation=False)
            last_loss = RMSEloss
            if verbose: print(f"---\nEpoch {epoch} - Loss: {last_loss:.3f}\n---")
    return last_loss

def one_epoch_eval(verbose=False):
    gnet.eval()
    running_validation_loss = 0.
    if verbose: print('Evaluation...')
    with torch.no_grad():
        for i, vdata in enumerate(test_loader):
            inputs, labels, idx = vdata
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = gnet(inputs)
            loss = loss_function(outputs, labels)
            RMSEloss =  np.sqrt(loss.item())
            running_validation_loss += RMSEloss
            gnet.log_the_loss(RMSEloss,epoch=False, validation = True)
    mean_vloss = gnet.log_the_loss(RMSEloss/(i+1), epoch=True, validation=True)
    if verbose: print(f"---\nValidation Loss: {mean_vloss:.3f}\n---")
    return mean_vloss
    

In [55]:
for epoch in range(0,2):
    print(f'Training epoch {epoch}')
    epoch_last_loss = one_epoch_train(verbose=True)
    #print(f'Epoch {epoch} - Loss: {epoch_last_loss:.3f}')
    epoch_vloss = one_epoch_eval(verbose=True)
    #print(f'Epoch {epoch} - Validation Loss: {epoch_vloss:.3f}')

Training epoch 0
Batch 1/385 - Loss: 0.280
Batch 11/385 - Loss: 0.287
Batch 21/385 - Loss: 0.270
Batch 31/385 - Loss: 0.278
Batch 41/385 - Loss: 0.284
Batch 51/385 - Loss: 0.291
Batch 61/385 - Loss: 0.273
Batch 71/385 - Loss: 0.280
Batch 81/385 - Loss: 0.280
Batch 91/385 - Loss: 0.277
Batch 101/385 - Loss: 0.285
Batch 111/385 - Loss: 0.287
Batch 121/385 - Loss: 0.272
Batch 131/385 - Loss: 0.283
Batch 141/385 - Loss: 0.284
Batch 151/385 - Loss: 0.283
Batch 161/385 - Loss: 0.283
Batch 171/385 - Loss: 0.287
Batch 181/385 - Loss: 0.289
Batch 191/385 - Loss: 0.284
Batch 201/385 - Loss: 0.281
Batch 211/385 - Loss: 0.279
Batch 221/385 - Loss: 0.284
Batch 231/385 - Loss: 0.279
Batch 241/385 - Loss: 0.284
Batch 251/385 - Loss: 0.271
Batch 261/385 - Loss: 0.284
Batch 271/385 - Loss: 0.279
Batch 281/385 - Loss: 0.274
Batch 291/385 - Loss: 0.269
Batch 301/385 - Loss: 0.278
Batch 311/385 - Loss: 0.278
Batch 321/385 - Loss: 0.280
Batch 331/385 - Loss: 0.272
Batch 341/385 - Loss: 0.276
Batch 351/385 

## Saving the model

In [56]:
#explore our model parameter so we don't have to retrain if we want to use it again
now = datetime.now().strftime("%d-%m_%H-%M-%S")
model_last_name = f'gnet/trained_gnet_{now}.pth'
torch.save({
    'model_state_dict' : gnet.state_dict(), # tutti i pesi
    'optimizer_state_dict' : optimizer.state_dict(), # values of the optimizer, loss is just the last loss value.
    'loss' : gnet.loss_dict,
    'batchsize_train_test' : (batch_size_train, batch_size_test),
    'device' : device,
    'is_rgb' : is_rgb,
    'crop_size' : csize,
    'kernel_size' : kernel_size,
    'out_channels' : out_channels,
    'feature_map_2' : feature_map_2,
    'max_pool_kernel' : max_pool_kernel,
    'loss_function' : str(loss_function),
    'optimizer' : str(optimizer),
    'epoch' : len(gnet.loss_dict['epoch'])}, model_last_name)

## Loading the model

In [ ]:
gnet = GalaxyNet()
gnet.load_state_dict(torch.load(model_last_name)['model_state_dict'])
optimizer.load_state_dict(torch.load(model_last_name)['optimizer_state_dict'])
loss = torch.load(model_last_name)['loss']




2

In [ ]:
def objective(trial):
    model = GalaxyNet().to(device)
    optimizer_ optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    train_loader, valid_loader = 

SyntaxError: incomplete input (3449955012.py, line 1)